### Input and Output with Serialization in LangChain and OpenAI

This program shows how to save and reload a setup for a language model (an AI that understands and generates text). It uses a tool called LangChain and OpenAI's API to:

1. Create a task where the AI translates English text into French.  
2. Save the setup (like a recipe) into a file so it can be reused later.  
3. Load the saved setup back from the file.  
4. Test the loaded setup by asking the AI to translate a sentence.  

It's like writing down a recipe, storing it, and then using it again later to cook the same dish!

In [7]:
# Read the open ai API key from your text file
f = open('C:\\Users\\Shailendra Kadre\\Desktop\\OPEN_AI_KEY.txt')
api_key = f.read()

In [8]:
import json
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableSequence

# Step 1: Provide your OpenAI API key
#api_key = "your_openai_api_key_here"  # Replace with your actual OpenAI API key

# Step 2: Initialize the ChatOpenAI model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=api_key)

# Step 3: Create a prompt template
prompt = PromptTemplate(
    input_variables=["text"],
    template="Translate this to French: {text}"
)

# Step 4: Create a RunnableSequence (replaces LLMChain)
chain = prompt | llm

# Step 5: Serialize the RunnableSequence manually
serialization_path = "llm_chain.json"
# Save the necessary configuration parameters for reconstruction
chain_dict = {
    "llm": {
        "model": "gpt-3.5-turbo",
        "temperature": 0,
        "openai_api_key": api_key
    },
    "prompt": {
        "template": prompt.template,
        "input_variables": prompt.input_variables
    }
}
with open(serialization_path, "w") as f:
    json.dump(chain_dict, f)
print("Serialized RunnableSequence saved to llm_chain.json.")

# Step 6: Load the serialized data
with open(serialization_path, "r") as f:
    chain_data = json.load(f)

# Step 7: Reconstruct the RunnableSequence manually
# Recreate the ChatOpenAI instance
loaded_llm = ChatOpenAI(
    model=chain_data["llm"]["model"],
    temperature=chain_data["llm"]["temperature"],
    openai_api_key=chain_data["llm"]["openai_api_key"]
)

# Recreate the PromptTemplate instance
loaded_prompt = PromptTemplate(
    input_variables=chain_data["prompt"]["input_variables"],
    template=chain_data["prompt"]["template"]
)

# Recreate the RunnableSequence
loaded_chain = loaded_prompt | loaded_llm
print("Loaded RunnableSequence successfully!")

# Step 8: Use the loaded chain to process input
input_text = {"text": "Hello, how are you?"}
output = loaded_chain.invoke(input_text)

# Extract only the main content from the output
if isinstance(output, dict) and "content" in output:
    output_text = output["content"]
else:
    output_text = output.content if hasattr(output, "content") else str(output)

# Step 9: Print concise output
print(f"Output: {output_text}")


Serialized RunnableSequence saved to llm_chain.json.
Loaded RunnableSequence successfully!
Output: Bonjour, comment vas-tu ?
